# Qhue experiments

Experiments with the [Qhue](https://github.com/quentinsf/qhue) python module.

If you haven't already, then `pip install qhue` before starting.  

Some of these examples may assume you have a recent bridge with recent software.

*If you're viewing this with my sample output, I've truncated some of it for readability. I have a lot of lights!*

## Basics 

In [50]:
# Put in the IP address of your Hue bridge here
BRIDGE_IP='192.168.1.185'
username='BykNjzzgLxSgbHS5AfCMkSiLFrAVG4mHxxm1yFTp'

from qhue import Bridge, QhueException, create_new_username
import time
from random import randint

In [51]:
bridge = Bridge(BRIDGE_IP, username)
lights = bridge.lights()
for num, info in lights.items():
    print("{:16} {}".format(info['name'], num))

Table            1
Bar              2
Hue color lamp 3 3
Hue color lamp 4 4


Let's get the numbers and names of the lights:

In [52]:
bridge.lights[3].state(on=False)

[{'success': {'/lights/3/state/on': False}}]

In [53]:
seq = [
    { 'light': 1, 'state': True, 'hue': randint(200, 65535), 'br': randint(0, 255) },
    { 'light': 2, 'state': True, 'hue': randint(4381, 65535), 'br': randint(0, 255) },
    { 'light': 1, 'state': True, 'hue': randint(9192, 65535), 'br': randint(0, 255) },
    { 'light': 4, 'state': True, 'hue': randint(7141, 65535), 'br': randint(0, 255) },
    { 'light': 2, 'state': True, 'hue': randint(0, 65535), 'br': randint(0, 255) },
    { 'light': 4, 'state': True, 'hue': randint(4000, 65535), 'br': randint(0, 255) },
]


In [54]:
for s in seq:
    bridge.lights[s['light']].state(on=s['state'], hue=s['hue'], bri=255)
    time.sleep(1)

In [ ]:
for i in ['1', '2', '3', '4']:
    bridge.lights[i].state(on=True)

In [ ]:
while True:
    for i in range(randint(3,10)):
        step = randint(1,10)/10.0
        bridge.lights[randint(1,2)].state(on=True, hue=randint(0, 65)*1000, bri=255)
        time.sleep(0.1)

In [34]:
light_states = {
    "1": {"on":True, "hue":52000},
    "2": {"on":True, "hue":5200}
}

for num, attributes in light_states.items():
    bridge.lights[num].state(on=attributes["on"], hue=attributes["hue"], bri=255)

In [41]:
while True:
    for i in range(randint(3,10)):
        step = randint(1,10)/10.0
        light_num = randint(1,2)
        new_hue = light_states[str(light_num)]["hue"] + randint(0,10)*(1000)*(randint(0,1)*2-1)
        bridge.lights[light_num].state(on=True, hue=new_hue, bri=255)
        light_states[light_num]["hue"] = new_hue
        time.sleep(step)

QhueException: invalid value, -1800,, for parameter, hue